In [145]:
import pandas as pd
import re

def parseFile(path):
    """
    
    """
    standard = re.compile("(\d+) +(\d+)\.(\d+) +([a-z0-9_^ ]*)(\(.*\)) += +([^ ]*) (.*[^<])* *<((\d).*\.(\d).*)>$")
    unfinished = re.compile("(\d+) +(\d+)\.(\d+) +([^\(]*)\((.*[^<])(.*)>$")
    resumed = re.compile("(\d+) +(\d+)\.(\d+) +<(.*)> +(.*)\) += +([^ ]*) (.*[^<])* *<((\d).*\.(\d).*)>$")
    exitcall = re.compile("(\d+) +(\d+)\.(\d+) +([a-z0-9_^ ]*)(\(.*\)) += +([^ ]*)$")
    data = []
    missed = []
    MAX_ROWS = 900099999999
    with open(path, 'r') as file:
        line = file.readline()
        i=0
        while line:
            i+=1
            if(i>MAX_ROWS):
                print("Missed: %s \n %s" % (str(len(missed)), missed))
                return data
            if(resumed.search(line)):
                entry = {'pid': int(resumed.search(line).group(1), 10),
                         'timestamps': int(resumed.search(line).group(2), 10),
                         'timestampms': int(resumed.search(line).group(3), 10),
                         'syscall': resumed.search(line).group(4),
                         'args': 'RESUME',
                         'result': resumed.search(line).group(6),
                         'error': resumed.search(line).group(7),
                         'duration': float(resumed.search(line).group(8))
                        }
                data.append(entry)
            elif(standard.search(line)):
                entry = {'pid': int(standard.search(line).group(1), 10),
                         'timestamps': int(standard.search(line).group(2), 10),
                         'timestampms': int(standard.search(line).group(3), 10),
                         'syscall': standard.search(line).group(4),
                         'args': parseArg(standard.search(line).group(5)),
                         'result': standard.search(line).group(6),
                         'error': standard.search(line).group(7),
                         'duration': float(standard.search(line).group(8))
                        }
                data.append(entry)
                
            elif(unfinished.search(line)):
                entry = {'pid': int(unfinished.search(line).group(1), 10),
                         'timestamps': int(unfinished.search(line).group(2), 10),
                         'timestampms': int(unfinished.search(line).group(3), 10),
                         'syscall': unfinished.search(line).group(4),
                         'args': parseArg(unfinished.search(line).group(5) + ')'),
                         'result': None,
                         'error': 'UNFINISHED',
                         'duration': -1
                        }
                data.append(entry)
            elif(exitcall.search(line)):
                entry = {'pid': int(exitcall.search(line).group(1), 10),
                         'timestamps': int(exitcall.search(line).group(2), 10),
                         'timestampms': int(exitcall.search(line).group(3), 10),
                         'syscall': exitcall.search(line).group(4),
                         'args': parseArg(exitcall.search(line).group(5)),
                         'result': exitcall.search(line).group(6),
                         'error': None,
                         'duration': None
                        }
                data.append(entry)
            else:
                missed.append(line)
                
            line = file.readline()
        #while
        print("Missed: %s \n %s" % (str(len(missed)), missed))
    return data

def parseArg(args):
    """
    """
    stack = ['(']
    listArgs = []
    pointer = listArgs
    currentArg = ""
    inQuote = False
    skip = False
    try:
        for char in args[1:]:

            if skip:
                skip = False
                continue
            elif char == '"':
                inQuote = not inQuote
                continue
            elif inQuote :

                currentArg += char
            else: # not in quotes
                bracket = stack.pop()
                if bracket == '(' and char == ')' or bracket == '[' and char == ']' or bracket == '{' and char == '}':
                    pointer.append(currentArg.strip())
                    currentArg = ""
                    pointer = listArgs
                    for i in range(len(stack)-1):
                        pointer = pointer[len(pointer)-1]
                    skip = True
                    continue
                else:
                    stack.append(bracket)
                    if char in ['(', '{', '[']:
                        stack.append(char)
                        pointer.append([])
                        pointer = pointer[len(pointer)-1]
                    elif char == ',':
                        pointer.append(currentArg.strip())
                        currentArg = ""
                    else:
                        currentArg += char
        return listArgs
    except:
        print("Error Parsing: %.50s ..." % args)
        return [args]

data = pd.DataFrame(parseFile("output-large.log"))
cols = ['pid', 'syscall', 'args', 'result', 'error', 'duration']
data.set_index(['timestamps', 'timestampms'], inplace = True)
data = data[cols]
data



Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (3, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (5, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (5, "\177ELF\2\1\1\0\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\ ...
Error Parsing: (10, "\211PNG\r\n\32\n\0\0\0\rIHDR\0\0\0\20\0\0\0\ ...
Error Parsing: (10, 

syscall  \
pid  timestamps timestampms                       
1919 1561668639 708112                   execve   
                708204                      brk   
                708223                   access   
                708244                   access   
                708262                     open   
                708277                    fstat   
                708293                     mmap   
                708305                    close   
                708316                   access   
                708332                     open   
                708348                     read   
                708366                    fstat   
                708379                     mmap   
                708393                     mmap   
                708406                 mprotect   
                708420                     mmap   
                708435                     mmap   
                708449                    close   
                708463                   access   
                708477                     open   
                708491                     read   
                708511                    fstat   
                708524                     mmap   
                708536                 mprotect   
                708548                     mmap   
                708564                    close   
                708577                   access   
                708594                     open   
                708607                     read   
                708625                    fstat   
...                                         ...   
1938 1561668684 985603                    close   
                985614                   munmap   
                985633                   access   
                985918                     stat   
                985942                     open   
                985970                    fstat   
                985982                     mmap   
                985999                    write   
                986372                    close   
                986412                   munmap   
                986454                    write   
                986476                    write   
                986503                     poll   
                986517                   writev   
1940 1561668684 995509         ... poll resumed   
                995529                     read   
1938 1561668684 999093       ... writev resumed   
                999127                     poll   
                999143                  recvmsg   
                999402                  recvmsg   
                999462                     open   
                999932                fallocate   
                999957                    write   
                999978                  fstatfs   
                999991                    lstat   
     1561668685 344                       fsync   
                639                       close   
                654                      rename   
                675                       chmod   
                827                  exit_group   

                                                                          args  \
pid  timestamps timestampms                                                      
1919 1561668639 708112          [/usr/bin/leafpad, [leafpad], [/* 52 vars */]]   
                708204                                                     [0]   
                708223                              [/etc/ld.so.nohwcap, F_OK]   
                708244                              [/etc/ld.so.preload, R_OK]   
                708262                  [/etc/ld.so.cache, O_RDONLY|O_CLOEXEC]   
                708277        [3, [st_mode=S_IFREG|0644, st_size=111518, ...]]   
                708293            [NULL, 111518, PROT_READ, MAP_PRIVATE, 3, 0]   
                708305                                                     [3]   
      

In [120]:
data.loc[data['duration'] == -1]

,,,syscall,args,result,error,duration
pid,timestamps,timestampms,,,,,
1919,1561668639,751820,poll,"[[fd=7, events=POLLIN], , 1, 25000 <unfinished...",None,UNFINISHED,-1.0
1920,1561668639,752320,write,"[, \1\0\0\0\0\0\0\0, 8 <unfinished ...]",None,UNFINISHED,-1.0
1919,1561668639,752338,futex,"[x7fab198ca440, FUTEX_WAIT_PRIVATE, 2, NULL <u...",None,UNFINISHED,-1.0
1920,1561668639,752351,futex,"[x7fab198ca440, FUTEX_WAKE_PRIVATE, 1 <unfinis...",None,UNFINISHED,-1.0
1919,1561668639,752556,futex,"[x7fab198aed70, FUTEX_WAIT_PRIVATE, 2, NULL <u...",None,UNFINISHED,-1.0
1920,1561668639,752591,futex,"[x7fab198aed70, FUTEX_WAKE_PRIVATE, 1 <unfinis...",None,UNFINISHED,-1.0
1919,1561668639,752784,poll,"[[fd=7, events=POLLIN], , 1, 25000 <unfinished...",None,UNFINISHED,-1.0
1920,1561668639,753313,write,"[, \1\0\0\0\0\0\0\0, 8 <unfinished ...]",None,UNFINISHED,-1.0
1919,1561668639,753330,futex,"[x7fab198cdcd0, FUTEX_WAIT_PRIVATE, 2, NULL <u...",None,UNFINISHED,-1.0


In [121]:
data.loc[data['args'] == 'RESUME']

,,,syscall,args,result,error,duration
pid,timestamps,timestampms,,,,,
1919,1561668639,752330,... poll resumed,RESUME,1,"([{fd=7, revents=POLLIN}])",0.000506
1920,1561668639,752345,... write resumed,RESUME,8,None,0.000021
1919,1561668639,752358,... futex resumed,RESUME,0,None,0.000017
1920,1561668639,752565,... futex resumed,RESUME,1,None,0.000211
1919,1561668639,752597,... futex resumed,RESUME,0,None,0.000037
1920,1561668639,752791,... futex resumed,RESUME,1,None,0.000198
1919,1561668639,753322,... poll resumed,RESUME,1,"([{fd=7, revents=POLLIN}])",0.000535
1920,1561668639,753336,... write resumed,RESUME,8,None,0.000019
1919,1561668639,753349,... futex resumed,RESUME,0,None,0.000016


In [158]:
df_new = data.loc[data['syscall'].isin(['clone', 'fork'])].copy()
df_new

syscall  \
pid  timestamps timestampms           
1919 1561668639 751656        clone   
                757490        clone   
                757802        clone   
                757869        clone   
     1561668660 125482        clone   
1930 1561668660 141987        clone   
1931 1561668660 189460        clone   
                193006        clone   
                193306        clone   
                193370        clone   
     1561668661 477443        clone   
1937 1561668661 477789        clone   
1938 1561668661 543600        clone   
                548467        clone   
                548765        clone   
                548834        clone   
1931 1561668669 59324         clone   
1938 1561668674 62462         clone   
1947 1561668674 62801         clone   
1948 1561668674 107682        clone   
                111216        clone   
                111507        clone   
                111580        clone   
1938 1561668679 715760        clone   
                715825        clone   

                                                                          args  \
pid  timestamps timestampms                                                      
1919 1561668639 751656       [child_stack=0x7fab10a18d30, flags=CLONE_VM|CL...   
                757490       [child_stack=0x7fab0bffed30, flags=CLONE_VM|CL...   
                757802       [child_stack=0x7fab0b7fdd30, flags=CLONE_VM|CL...   
                757869       [child_stack=0x7fab0affcd30, flags=CLONE_VM|CL...   
     1561668660 125482       [child_stack=0, flags=CLONE_CHILD_CLEARTID|CLO...   
1930 1561668660 141987       [child_stack=0, flags=CLONE_CHILD_CLEARTID|CLO...   
1931 1561668660 189460       [child_stack=0x7feb93583d30, flags=CLONE_VM|CL...   
                193006       [child_stack=0x7feb92d82d30, flags=CLONE_VM|CL...   
                193306       [child_stack=0x7feb92581d30, flags=CLONE_VM|CL...   
                193370       [child_stack=0x7feb91d80d30, flags=CLONE_VM|CL...   
     1561668661 477443       [child_stack=0, flags=CLONE_CHILD_CLEARTID|CLO...   
1937 1561668661 477789       [child_stack=0, flags=CLONE_CHILD_CLEARTID|CLO...   
1938 1561668661 543600       [child_stack=0x7fbabe171d30, flags=CLONE_VM|CL...   
                548467       [child_stack=0x7fbabd970d30, flags=CLONE_VM|CL...   
                548765       [child_stack=0x7fbabd16fd30, flags=CLONE_VM|CL...   
                548834       [child_stack=0x7fbabc96ed30, flags=CLONE_VM|CL...   
1931 1561668669 59324                                        [<unfinished ...]   
1938 1561668674 62462        [child_stack=0, flags=CLONE_CHILD_CLEARTID|CLO...   
1947 1561668674 62801        [child_stack=0, flags=CLONE_CHILD_CLEARTID|CLO...   
1948 1561668674 107682       [child_stack=0x7f62ee320d30, flags=CLONE_VM|CL...   
                111216       [child_stack=0x7f62edb1fd30, flags=CLONE_VM|CL...   
                111507       [child_stack=0x7f62ed31ed30, flags=CLONE_VM|CL...   
                111580       [child_stack=0x7f62ecb1dd30, flags=CLONE_VM|CL...   
1938 1561668679 715760       [child_stack=0x7fbabd16fd30, flags=CLONE_VM|CL...   
                715825       [child_stack=0x7fbabc96ed30, flags=CLONE_VM|CL...   

                            result       error  duration  
pid  timestamps timestampms                               
1919 1561668639 751656        1920        None  0.000012  
                757490        1921        None  0.000012  
                757802        1922        None  0.000009  
                757869        1923        None  0.000009  
     1561668660 125482        1930        None  0.000197  
1930 1561668660 141987        1931        None  0.000221  
1931 1561668660 189460        1932        None  0.000012  
                193006        1933        None  0.000010  
                193306        1934        None  0.000009  
                193370        1935        None  0.000009  
     1561668661 477443        1937        None  0.0